# Building a Spam Filter with Naive Bayes

In this project, I am going to use the multinomial Naive Bayes algorithm to build a spam filter for SMS messages. The data I use in this project is from as put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection).

For this project, my goal is to create a spam filter that classifies new messages with an accuracy greater than 80% — so it's expected that more than 80% of the new messages will be classified correctly as spam or ham (non-spam).

### Summary of results
A spam filter for SMS messages using the multinomial Naive Bayes algorithm is created. The filter had an accuracy of 98.74% on the test set.

## Reading in data set

In [1]:
import pandas as pd

# read in data set
data = pd.read_csv('SMSSpamCollection', sep='\t',header=None, names=['Label', 'SMS'])

# quick exploration of the dataset
print('shape: ',data.shape)

shape:  (5572, 2)


In [2]:
# first 3 rows
print(data.head(3))

  Label                                                SMS
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...


ham means non-spam.

In [3]:
# percentage of spam message
data['Label'].value_counts(normalize = True) * 100

ham     86.593683
spam    13.406317
Name: Label, dtype: float64

About 13% of the messages in this data set are spam.

## Creating training set and test set

I am going to keep 80% of our dataset for training, and 20% for testing. The dataset has 5,572 messages, which means that:

- The training set will have 4,458 messages (about 80% of the dataset).
- The test set will have 1,114 messages (about 20% of the dataset).

In [4]:
# randomize the entire dataset.
data_randomized = data.sample(frac=1, random_state=1)

# calculate index for training data
training_index = int(round(len(data_randomized) * 0.8, 0))

# extract training data and reset index
training_set = data_randomized[:training_index].reset_index(drop = True)
print(training_set.shape)
print(training_set['Label'].value_counts(normalize = True) * 100)
print('\n')

# extract test data and reset index
test_set = data_randomized[training_index:].reset_index(drop = True)
print(test_set.shape)
print(test_set['Label'].value_counts(normalize = True) * 100)

(4458, 2)
ham     86.54105
spam    13.45895
Name: Label, dtype: float64


(1114, 2)
ham     86.804309
spam    13.195691
Name: Label, dtype: float64


The data set is splited into training set and test set. Their percentage of spam message are similar.

## Data Cleaning

To calculate all these probabilities, I'll first need to perform a bit of data cleaning to bring the data in a format that will allow me to extract easily all the information I need.

### Removing punctation and changing letters to lower case

First, I will remove all the punctation and convert letters to lower case.

In [5]:
#before cleaning
training_set.head()

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [6]:
# remove punctation 
training_set['SMS'] = training_set['SMS'].str.replace('\W', ' ')

# transform every letter in every word to lower case
training_set['SMS'] = training_set['SMS'].str.lower()

In [7]:
# after removing punctation and transforming letters to lower case
training_set.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


### Creating Vocabulary list

Then a list will be created to contain all the word in the SMS

In [8]:
# transform SMS to list of works
training_set['SMS'] = training_set['SMS'].str.split()

# create empty list
vocabulary = []

# add SMS words into the list
for msg in training_set['SMS']:
    for word in msg:
        vocabulary.append(word)

# change list to set to remove deplicates and change back to list
vocabulary = list(set(vocabulary))
len(vocabulary)

7783

There are 7783 words in the vacabulary list.

### Creating DataFrame

The vocabulary list will be used to create a DateFrame which makes the calculations of the filter easier.

In [9]:
# create a dictionary

word_counts_per_sms = {
    unique_word: [0] * len(training_set['SMS']) for unique_word in vocabulary}

# loop over the sms and count the words
for index, sms in enumerate(training_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1
        
# transform dictionary to dataframe
word_counts_per_sms = pd.DataFrame(word_counts_per_sms)

word_counts_per_sms.head()

,0,00,000,000pes,008704050406,0089,01223585334,02,0207,02072069400,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


In [10]:
# Concatenate the DataFrame with the traning set
training_set_clean = pd.concat([training_set, word_counts_per_sms], axis = 1)
training_set_clean.head()

,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,02,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


## Calculating constant variables

Now I have a clean data set. I am starting to build the spam filter.

For the Naive Bayes algorithm, the probability values of the two equations below are needed to be able to classify new messages:

$$
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam)
$$$$
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
$$

Also, to calculate P(wi|Spam) and P(wi|Ham) inside the formulas above, I need to use these equations:
$$
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
$$$$
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
$$

I will start by calculating:
- P(Spam) and P(Ham)
- N<sub>Spam</sub>, N<sub>Ham</sub>, N<sub>Vocabulary</sub>

In this project, Laplace smoothing is used and set $\\alpha = 1$.

In [11]:
# sperate dataframe to spam messages and ham messages
spam_msg = training_set_clean[training_set_clean['Label'] == 'spam']
ham_msg = training_set_clean[training_set_clean['Label'] == 'ham']

# P(Spam)
p_spam = len(spam_msg)/len(training_set_clean)

# P(Ham)
p_ham = len(ham_msg)/len(training_set_clean)

# N_spam
n_spam = spam_msg['SMS'].apply(len).sum()

# N_ham
n_ham = ham_msg['SMS'].apply(len).sum()

# N_vocabulary
n_vocab = len(vocabulary)

# laplace smoothing
alpha = 1

## Calculating Parameters

Now I will be calculating the parameters by using the following formulars.
$$
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
$$$$
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
$$

In [12]:
# create empty parameter dictionaries for spam and ham
parameter_spam = {unique_word: 0 for unique_word in vocabulary}
parameter_ham = {unique_word: 0 for unique_word in vocabulary}

# calculate the parameters and update to the dictionaries
for word in vocabulary:
    n_word_given_spam = spam_msg[word].sum()
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha * n_vocab)
    parameter_spam[word] = p_word_given_spam
    
    n_word_given_ham = ham_msg[word].sum()
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha * n_vocab)
    parameter_ham[word] = p_word_given_ham

## Creating spam filtering function

Now I have all the constants and parameters, I can start creating the spam filtering function. The spam filter can be understood as a function that:

- Takes in as input a new message (w1, w2, ..., wn)
- Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn)
- Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and:
    - If P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then the message is classified as ham.
    - If P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then the message is classified as spam.
    - If P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then the algorithm may request human help.

In [13]:
import re

def classify(message):
    '''
    
    message: a string
    
    '''

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in parameter_spam:
            p_spam_given_message *= parameter_spam[word]
        if word in parameter_ham:
            p_ham_given_message *= parameter_ham[word]

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classification.')

In [14]:
# test 1
msg_1 = 'WINNER!! This is the secret code to unlock the money: C3421.'
classify(msg_1)

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


In [15]:
# test 2
msg_2 = "Sounds good, Tom, then see u there"
classify(msg_2)

P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


## Testing function with test set

Now I will use the test set to test the accuracy of the function. Accuracy is determined by number of correctly classified messages divided by total number of classified messages.

First, I am change the outout of the function to returning 'ham' or 'spam.

In [19]:
def classify_test_set(message):
    '''
    message: a string
    '''

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameter_spam:
            p_spam_given_message *= parameter_spam[word]

        if word in parameter_ham:
            p_ham_given_message *= parameter_ham[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [20]:
# add new column for the output of the function
test_set['predicted'] = test_set['SMS'].apply(classify_test_set)
test_set.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


In [36]:
# measure the accuracy
correct = 0
total = len(test_set)

for row in test_set.iterrows():
    row = row[1]
    if row['Label'] == row['predicted']:
        correct += 1

accuracy = correct / total
print('Correct: ', correct)
print('Incorrect: ', total - correct)
print('Accuracy %: ', accuracy * 100)

Label                                              ham
SMS          Later i guess. I needa do mcat study too.
predicted                                          ham
Name: 0, dtype: object

The accuracy of the filter is 98.74%.

## Conclusion

In this project, I have created a spam filter for SMS messages using the multinomial Naive Bayes algorithm. The filter had an accuracy of 98.74% on the test set.

## Further study

- Isolate the 14 messages that were classified incorrectly and try to figure out why the algorithm reached the wrong conclusions.
- Make the filtering process more complex by making the algorithm sensitive to letter case.